In [12]:
#Installing dependencies and detectron
!pip install torch torchvision opencv-python
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 228.7 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 191.5 MB/s  0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [opencv-python]0m [opencv-python]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
amazon-sagemaker-jupyter-ai-q-developer 1.2.7 requires onnxruntime<2,>=1.15.0, which is not installed.
autogluon-multimodal 1.4.0 requires nvidia-ml-py3<8.0,>=7.352.0, which is not installed.
amazon-sagemaker-jupyter-ai-q-developer 1.2.7 requires numpy<=2.0.1, but you have numpy 2.2.6 which is incompatible.
amazon-sagemaker-sql-magic 0.1.4 requires numpy<2, but you have numpy 2.2.6 which is incompatible.
autogluon-multimodal 1

In [11]:
import os
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog
import warnings
import torch
from scripts import tools
from scripts.visualize import BatchVisualizer as bv
from scripts import custom_transforms as CT
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'cv2'

In [3]:
#loading paths and annotations
root = "./Some-Trees-Dataset-10"
train_ann = os.path.join(root, "train", "_annotations_fixed.coco.json")
val_ann = os.path.join(root, "valid", "_annotations_fixed.coco.json")
test_ann = os.path.join(root, "test", "_annotations_fixed.coco.json")

In [4]:
#loading images
train_img = os.path.join(root,"train")
val_img = os.path.join(root,"valid")
test_img = os.path.join(root,"test")

In [5]:
classes = ["tree"]

register_coco_instances("trees_train", {}, train_ann, train_img)
register_coco_instances("trees_val", {}, val_ann, val_img)
register_coco_instances("trees_test", {}, test_ann, test_img)

# register thing classes to metadata (optional, but used by Visualizer)
MetadataCatalog.get("trees_train").set(thing_classes=classes)
MetadataCatalog.get("trees_val").set(thing_classes=classes)
MetadataCatalog.get("trees_test").set(thing_classes=classes)

namespace(name='trees_test',
          json_file='./Some-Trees-Dataset-10/test/_annotations_fixed.coco.json',
          image_root='./Some-Trees-Dataset-10/test',
          evaluator_type='coco',
          thing_classes=['tree'])

In [6]:
print(MetadataCatalog.get("trees_train").thing_classes)

['tree']


In [7]:
#train with detectron2
import os
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo

output_dir = "./detectron2_output"

In [8]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("trees_train",)
cfg.DATASETS.TEST = ("trees_val",)  # used by evaluator
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

In [9]:
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 6.25e-4         # good scaled LR for batch 4
cfg.SOLVER.MAX_ITER = 20000          # ≈ 61 epochs
cfg.SOLVER.STEPS = [14000, 18000]    # decay LR twice near the end
cfg.SOLVER.CHECKPOINT_PERIOD = 2000  # save every ~6 epochs
cfg.TEST.EVAL_PERIOD = 1000          # run AP eval every ~3 epochs
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
cfg.OUTPUT_DIR

'./output'

In [10]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[09/26 13:53:56 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_f10217.pkl: 178MB [00:01, 177MB/s]                               
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due t

[09/26 13:53:58 d2.engine.train_loop]: Starting training from iteration 0
[09/26 13:54:13 d2.utils.events]:  eta: 3:31:20  iter: 19  total_loss: 1.567  loss_cls: 0.5261  loss_box_reg: 0.1111  loss_mask: 0.69  loss_rpn_cls: 0.2075  loss_rpn_loc: 0.02135    time: 0.6392  last_time: 0.5832  data_time: 0.0155  last_data_time: 0.0072   lr: 1.2488e-05  max_mem: 3097M


2025-09-26 13:54:14.034163: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-26 13:54:14.054540: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758894854.075743    4508 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758894854.082592    4508 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-26 13:54:14.099978: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

[09/26 13:54:31 d2.utils.events]:  eta: 3:33:12  iter: 39  total_loss: 1.416  loss_cls: 0.3914  loss_box_reg: 0.1527  loss_mask: 0.6823  loss_rpn_cls: 0.1652  loss_rpn_loc: 0.02341    time: 0.6494  last_time: 0.6862  data_time: 0.0072  last_data_time: 0.0071   lr: 2.4976e-05  max_mem: 3097M
[09/26 13:54:39 d2.engine.hooks]: Overall training speed: 50 iterations in 0:00:32 (0.6513 s / it)
[09/26 13:54:39 d2.engine.hooks]: Total training time: 0:00:38 (0:00:05 on hooks)
[09/26 13:54:39 d2.utils.events]:  eta: 3:33:04  iter: 52  total_loss: 1.358  loss_cls: 0.3227  loss_box_reg: 0.1535  loss_mask: 0.6728  loss_rpn_cls: 0.1143  loss_rpn_loc: 0.02242    time: 0.6465  last_time: 0.5715  data_time: 0.0072  last_data_time: 0.0068   lr: 3.2468e-05  max_mem: 3097M


KeyboardInterrupt: 